## Baseline Models

The aim of this script is to generate baseline models which use pretarined models like BERT , T5 and gpt2 and compare the performance of it vs a trained openprompt model. 

## Imports and Datapreprocessing 

In [1]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score,f1_score,classification_report
from sklearn.model_selection import train_test_split
import transformers
from transformers import BertModel, BertTokenizer
import warnings
import redditcleaner
from torch.utils.data import Dataset, DataLoader
import redditcleaner
!pip uninstall preprocessor
!pip install tweet-preprocessor==0.6.0
from transformers import AutoTokenizer
from tqdm.contrib import tenumerate

In [2]:
path=r"C:\Users\py22715\OneDrive - University of Bristol\Documents\Python Scripts"
df=pd.read_csv(os.path.join(path,"golden_set2.csv"))

In [3]:
df.head(5)

,id,subreddit,created_date,context,is_violation,violation,sentence
0,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets..."
1,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets..."
2,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,Incivility,"""Hey douchebag, use the NFL as your example o..."
3,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr..."
4,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr..."


### Defining functions for cleaning data 

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def reddit_clean(x):
            
            return tokenizer.tokenize(x)
def reddit_batch_clean(x):
            return [tokenizer.tokenize(redditcleaner.clean(e)) for e in x]


In [5]:
sentences=df['sentence']
#df.drop(['sentence'],axis=1,inplace= True)
final_sentences=[]
for i in sentences:
    final_sentences.append(reddit_clean(i))    
df['final_sentences']=final_sentences    
    

Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors


In [6]:
df['final_sentences']=final_sentences  
df.head()


,id,subreddit,created_date,context,is_violation,violation,sentence,final_sentences
0,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets...","["", It, ', s, worse, to, be, passive, -, aggre..."
1,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets...","["", It, ', s, worse, to, be, passive, -, aggre..."
2,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,Incivility,"""Hey douchebag, use the NFL as your example o...","["", Hey, do, ##uche, ##bag, ,, use, the, NFL, ..."
3,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr...","["", This, coming, from, a, sport, that, resolv..."
4,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr...","["", This, coming, from, a, sport, that, resolv..."


In [7]:
context=df['context']
final_context=[]
for i in context:
    final_context.append(reddit_clean(i))
df['final_context']=final_context  
df.head()

,id,subreddit,created_date,context,is_violation,violation,sentence,final_sentences,final_context
0,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets...","["", It, ', s, worse, to, be, passive, -, aggre...","[[, ', "", I, wasn, \, ', t, particularly, taug..."
1,813yb,AskReddit,2009-03-01 00:43:33,"[' ""I wasn\'t particularly taught that being p...",No,No Violations,"""It's worse to be passive-aggressive. It gets...","["", It, ', s, worse, to, be, passive, -, aggre...","[[, ', "", I, wasn, \, ', t, particularly, taug..."
2,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,Incivility,"""Hey douchebag, use the NFL as your example o...","["", Hey, do, ##uche, ##bag, ,, use, the, NFL, ...","[[, ', "", Yeah, ,, Don, Cherry, is, going, ove..."
3,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr...","["", This, coming, from, a, sport, that, resolv...","[[, ', "", Yeah, ,, Don, Cherry, is, going, ove..."
4,819lh,sports,2009-03-01 21:38:40,"[' ""Yeah, Don Cherry is going over the line he...",Yes,harrassment,"""This coming from a sport that resolves it pr...","["", This, coming, from, a, sport, that, resolv...","[[, ', "", Yeah, ,, Don, Cherry, is, going, ove..."


In [10]:
encoder = LabelEncoder()
df["label"]=encoder.fit_transform(df["violation"])
df['sentence']=df['context']+df['sentence']

train_text, val_text, train_labels, val_labels = train_test_split(np.array(df['sentence']).flatten().tolist(), df['label'], 
                                                                    random_state=2018, 
                                                                    train_size=0.8 
                                                                    )
len(train_text)

173

In [11]:
tokens_train = tokenizer.batch_encode_plus(
   list(train_text),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
   list(val_text),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

C:\Users\py22715\.conda\envs\v38\lib\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])

In [13]:
val_y = torch.tensor(list(val_labels))
train_y = torch.tensor(train_labels.tolist())

In [14]:
bert = BertModel.from_pretrained("bert-base-cased",return_dict=False)
for param in bert.parameters():
    param.requires_grad = False

In [15]:
import torch.nn as nn
class BERT_Arch(nn.Module):
    def __init__(self, bert,label_map):
        super(BERT_Arch, self).__init__()
        self.bert = bert 
      
        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,len(label_map))

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

        #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)
        return x

In [16]:
label_map=le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
model = BERT_Arch(bert,label_map)

# push the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [17]:
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-5)
loss_func = torch.nn.CrossEntropyLoss()
epochs = 5
batch=16

C:\Users\py22715\.conda\envs\v38\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
def train():
    model.train()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
    total_labels =[]
  
    # iterate over batches
    for step,batch in tenumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 5000 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
        model.zero_grad()        
        preds = model(sent_id,mask)
        loss = loss_func(preds, labels)
        total_loss = total_loss + loss.item()

        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds = preds.detach().cpu().numpy()
        preds = np.argmax(preds, axis=1)
        # append the model predictions
        total_preds+=list(preds)
        total_labels+=labels.tolist()

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    #total_preds  = np.concatenate(total_preds, axis=0)
    f1 = f1_score(total_labels, total_preds, average='micro')
    #returns the loss and predictions
    return avg_loss, f1

In [19]:
def evaluate():
  
    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []
    total_labels = []
    # iterate over batches
    for step,batch in tenumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 5000 == 0 and not step == 0:

          # Calculate elapsed time in minutes.
          #elapsed = format_time(time.time() - t0)

          # Report progress.
          print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = loss_func(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()
            preds = np.argmax(preds, axis=1)
            total_preds+=list(preds)
            total_labels+=labels.tolist()
    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    #total_preds  = np.concatenate(total_preds, axis=0)
    
    f1 = f1_score(total_labels, total_preds, average='micro')
    return avg_loss, f1

In [20]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data)
print(len(train_dataloader))

173


In [21]:
import numpy as np
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, f1_train = train()
    print("Training loss and F1")
    print(train_loss)
    print(f1_train)
    #evaluate model
    valid_loss, f1_valid = evaluate()
    print("Training loss and F1")
    print(train_loss)
    print(f1_train)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        file_name = 'topic_saved_weights.pt'
        #save_checkpoint(file_name, epoch, model, optimizer, label_map, id2label)
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')
    print(f'\nTraining F1: {f1_train:.3f}')
    print(f'Validation F1: {f1_valid:.3f}')


 Epoch 1 / 5


  0%|          | 0/173 [00:00<?, ?it/s]

Training loss and F1
1.2505881147233049
0.6184971098265896

Evaluating...


  0%|          | 0/44 [00:00<?, ?it/s]

Training loss and F1
1.2505881147233049
0.6184971098265896

Training Loss: 1.251
Validation Loss: 0.975

Training F1: 0.618
Validation F1: 0.682

 Epoch 2 / 5


...

Training loss and F1
1.687546156465232
0.6184971098265896

Training Loss: 1.688
Validation Loss: 1.370

Training F1: 0.618
Validation F1: 0.682

 Epoch 5 / 5


  0%|          | 0/173 [00:00<?, ?it/s]

Training loss and F1
1.725835507660243
0.6184971098265896

Evaluating...


  0%|          | 0/44 [00:00<?, ?it/s]

Training loss and F1
1.725835507660243
0.6184971098265896

Training Loss: 1.726
Validation Loss: 1.388

Training F1: 0.618
Validation F1: 0.682
